# Solving an optimization problem with the framework

In [10]:
import metaheuristic_designer as mhd
from metaheuristic_designer import ObjectiveVectorFunc
from metaheuristic_designer.initializers import UniformVectorInitializer
from metaheuristic_designer.searchMethods import GeneralSearch
from metaheuristic_designer.algorithms import HillClimb, GA, CRO
from metaheuristic_designer.operators import OperatorInt
from metaheuristic_designer.selectionMethods import SurvivorSelection, ParentSelection
import numpy as np

## The objective function
To start, we need to define the function we want to optimize.

This is done by creating a class that inherits from ```ObjectiveFunc```, or in this case ```ObjectiveVectorFunc``` since it provides the framework to create objective functions that take vectors as inputs.

For this example, we will try to optimize the path on a grid with randomly placed treasures that the agent must collect. The higher the number of treasures collected, the higher the reward.

In [11]:
class TresureHunt(ObjectiveVectorFunc):
    """
    Our objective function function.

    Takes an integer-valued vector and outputs the number of treasures collected.
    """

    def __init__(self, n_moves, n_tresures, grid_size=(10, 10), name: str = "Tresure Hunt"):
        self.n_moves = n_moves
        self.n_tresures = n_tresures
        self.grid_size = np.array(grid_size)

        # We define a grid with n_treasures
        grid_flat = np.zeros(grid_size).flatten()
        grid_flat[:n_tresures] = 1
        np.random.shuffle(grid_flat)
        self.grid = grid_flat.reshape(grid_size)

        # We define some extra information for displaying the grid
        self.grid_print_dict = {0: " _", 1: " x", 2: " o", 3: " #"}

        # We specify the size of the vector (n_moves), wether to maximize or minimize (mode),
        # lower limit (low_lim), upper limit (up_lim) and name of the function (name).
        super().__init__(n_moves, mode="max", low_lim=0, up_lim=3, name=name)

    def objective(self, move_vector):
        """
        Go through the specified path and count the amount of treasures collected
        """

        # Set of moves the agent can take, accessed from 0 to 3
        move_set = np.array([[0, 1], [1, 0], [0, -1], [-1, 0]])

        # Inital position
        pos = np.array([0, 0])

        # Accumulator of treasures collected
        collected = 0

        # Copy of the grid
        grid_copy = self.grid.copy()

        # Travel the path specified by the vector
        for move in move_vector:
            # Move to the next position
            new_pos = pos + move_set[move]

            # Check if it is on bounds
            if new_pos[0] >= 0 and new_pos[0] < self.grid_size[0] and new_pos[1] >= 0 and new_pos[1] < self.grid_size[1]:
                pos = new_pos

                # Collect the treasure if there is one
                if grid_copy[*pos] == 1:
                    collected += 1
                    grid_copy[*pos] = 0

        return collected

    def display(self):
        """
        Display the grid as text.
        """

        grid_str = ""
        for row in self.grid:
            for cell in row:
                grid_str += self.grid_print_dict[cell]
            grid_str += "\n"
        print(grid_str)

    def display_solution(self, move_vector):
        """
        Display the grid with the path taken.
        """

        collected = 0
        move_set = np.array([[0, 1], [1, 0], [0, -1], [-1, 0]])
        pos = np.array([0, 0])
        grid_copy = self.grid.copy()

        if grid_copy[*pos] == 0:
            grid_copy[*pos] = 2
        elif grid_copy[*pos] == 1:
            grid_copy[*pos] = 3

        for move in move_vector:
            new_pos = pos + move_set[move]
            if new_pos[0] >= 0 and new_pos[0] < self.grid_size[0] and new_pos[1] >= 0 and new_pos[1] < self.grid_size[1]:
                pos = new_pos
                if grid_copy[*pos] == 0:
                    grid_copy[*pos] = 2
                elif grid_copy[*pos] == 1:
                    grid_copy[*pos] = 3

        grid_str = ""
        for row in grid_copy:
            for cell in row:
                grid_str += self.grid_print_dict[cell]
            grid_str += "\n"

        print(grid_str)

In [12]:
# Define the objective function
objfunc = TresureHunt(n_moves=200, n_tresures=40, grid_size=(20, 20))
print("The map we use:")
objfunc.display()

The map we use:
 _ _ _ _ _ _ _ _ _ _ _ _ x _ _ _ x _ _ _
 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 _ _ _ _ _ _ _ x _ _ _ _ _ _ _ x _ _ x _
 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 _ _ _ _ x _ _ _ _ _ _ _ _ _ _ _ _ _ x _
 _ _ _ _ _ _ _ _ _ _ _ _ _ x _ _ x _ _ _
 _ x _ _ _ _ _ _ x x _ _ _ _ _ _ x _ _ _
 x _ x _ _ _ x x _ _ _ _ _ _ _ _ x _ _ _
 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ x _ x _
 _ _ _ _ x _ _ _ _ _ _ _ _ _ _ _ x _ x _
 _ _ _ _ x x _ _ _ _ _ _ _ _ _ _ _ _ _ _
 _ _ _ _ _ _ _ _ _ _ x x _ _ _ _ _ _ x _
 _ _ _ _ x _ _ _ _ _ _ _ _ _ x _ x x _ _
 _ _ _ _ _ x _ _ _ _ _ _ _ _ _ _ _ _ _ _
 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 _ _ _ x x x _ _ _ _ _ _ _ _ _ _ _ _ _ _
 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 _ _ x _ _ _ _ _ x _ _ _ _ _ _ _ x _ _ _
 _ _ _ _ _ _ _ _ _ x _ _ _ _ _ _ _ _ _ _



In [13]:
params = {"stop_cond": "neval", "neval": 50000, "verbose": True, "v_timer": 2}

# Define a population initialization module
pop_init = UniformVectorInitializer(objfunc.vecsize, objfunc.low_lim, objfunc.up_lim, pop_size=1, dtype=int)

# Define the operators to use
mutate_op = OperatorInt("MutSample", {"method": "Uniform", "Low": 0, "Up": 3, "N": 8})

# Instanciate the algorithm
algorithm = HillClimb(pop_init, mutate_op)

# Define a Search method
search = GeneralSearch(objfunc, algorithm, params)

# Optimize the objective function
best_solution, best_fitness = search.optimize()

Initializing optimization of Tresure Hunt using HillClimb
---------------------------------------------------------

Optimizing Tresure Hunt using HillClimb:
	Real time Spent: 0.0 s
	CPU time Spent:  0.0 s
	Generation: 0
	Best fitness: 5
	Evaluations of fitness: 1

Optimizing Tresure Hunt using HillClimb:
	Real time Spent: 2.0 s
	CPU time Spent:  2.0 s
	Generation: 4376
	Best fitness: 16
	Evaluations of fitness: 4377

Optimizing Tresure Hunt using HillClimb:
	Real time Spent: 4.0 s
	CPU time Spent:  4.0 s
	Generation: 8630
	Best fitness: 19
	Evaluations of fitness: 8631

Optimizing Tresure Hunt using HillClimb:
	Real time Spent: 6.0 s
	CPU time Spent:  6.0 s
	Generation: 12992
	Best fitness: 20
	Evaluations of fitness: 12993

Optimizing Tresure Hunt using HillClimb:
	Real time Spent: 8.0 s
	CPU time Spent:  8.0 s
	Generation: 17345
	Best fitness: 20
	Evaluations of fitness: 17346

Optimizing Tresure Hunt using HillClimb:
	Real time Spent: 10.0 s
	CPU time Spent:  10.0 s
	Generation: 21

In [14]:
print(f"Number of treasures obtained: {best_fitness}")
objfunc.display_solution(best_solution)

Number of treasures obtained: 21
 o o o o _ _ _ _ _ _ _ _ x _ _ _ x _ _ _
 _ _ o o o _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 o o o o o o o # _ _ _ _ _ _ _ x _ _ x _
 _ o o o o o _ _ _ _ _ _ _ _ _ _ _ _ _ _
 o o o _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 o o o o # o o _ _ _ _ _ _ _ _ _ _ _ x _
 o o o o o o _ _ _ _ _ _ _ x _ _ x _ _ _
 o # _ _ o _ _ o # # _ _ _ _ _ _ x _ _ _
 # o # o _ _ # # _ o o _ _ _ _ _ x _ _ _
 _ _ _ o o _ o _ _ o o _ _ _ _ _ x _ x _
 _ _ o o # o o o o _ o o _ _ _ _ x _ x _
 _ _ o o # # o _ o o o o _ _ _ _ _ _ _ _
 _ _ _ _ o o o o o _ # # _ _ _ _ _ _ x _
 _ _ _ _ # o o o o o o o _ _ x _ x x _ _
 _ _ o o o # o o o o _ _ _ _ _ _ _ _ _ _
 _ _ _ o _ _ _ _ o o _ _ _ _ _ _ _ _ _ _
 _ _ _ # # # o o _ _ _ _ _ _ _ _ _ _ _ _
 _ _ _ _ _ _ _ o o _ _ _ _ _ _ _ _ _ _ _
 _ _ x _ o o o o # _ _ _ _ _ _ _ x _ _ _
 _ _ _ _ _ _ o o o # _ _ _ _ _ _ _ _ _ _



In [15]:
# Define the parameters of the optimization process
params = {"stop_cond": "neval", "neval": 50000, "verbose": True, "v_timer": 2}
pop_size = 100

# Define a population initialization module
pop_init = UniformVectorInitializer(objfunc.vecsize, objfunc.low_lim, objfunc.up_lim, pop_size=pop_size, dtype=int)

# Define the operators to use
mutate_op = OperatorInt("MutSample", {"method": "Uniform", "Low": 0, "Up": 3, "N": 8})
cross_op = OperatorInt("Multipoint")

# Define the parent selection method
parent_sel = ParentSelection("Tournament", {"amount": 3, "p": 1})

# Define the survivor selection method
surv_sel = SurvivorSelection("KeepBest")

# Instanciate the algorithm
algorithm = GA(pop_init, mutate_op, cross_op, parent_sel, surv_sel, {"pmut": 0.1, "pcross": 0.8})

# Define a Search method
search = GeneralSearch(objfunc, algorithm, params)

# Optimize the objective function
best_solution, best_fitness = search.optimize()

Initializing optimization of Tresure Hunt using GA
--------------------------------------------------

Optimizing Tresure Hunt using GA:
	Real time Spent: 0.04 s
	CPU time Spent:  0.04 s
	Generation: 0
	Best fitness: 9
	Evaluations of fitness: 100
	diversity: 0.955

Optimizing Tresure Hunt using GA:
	Real time Spent: 2.01 s
	CPU time Spent:  2.01 s
	Generation: 49
	Best fitness: 19
	Evaluations of fitness: 5000
	diversity: 1.03

Optimizing Tresure Hunt using GA:
	Real time Spent: 4.02 s
	CPU time Spent:  4.02 s
	Generation: 95
	Best fitness: 23
	Evaluations of fitness: 9600
	diversity: 1.07

Optimizing Tresure Hunt using GA:
	Real time Spent: 6.05 s
	CPU time Spent:  6.05 s
	Generation: 142
	Best fitness: 23
	Evaluations of fitness: 14300
	diversity: 1.07

Optimizing Tresure Hunt using GA:
	Real time Spent: 8.06 s
	CPU time Spent:  8.06 s
	Generation: 188
	Best fitness: 23
	Evaluations of fitness: 18900
	diversity: 1.07

Optimizing Tresure Hunt using GA:
	Real time Spent: 10.1 s
	CPU t

In [16]:
print(f"Number of treasures obtained: {best_fitness}")
objfunc.display_solution(best_solution)

Number of treasures obtained: 25
 o o _ _ _ _ _ _ _ _ _ _ x _ _ _ x _ _ _
 _ o o _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 _ _ o _ _ _ _ x _ _ _ _ _ _ _ x _ _ x _
 _ o o _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 o o o _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 o _ _ _ x _ _ o o o _ _ _ _ _ _ _ _ x _
 o o _ _ _ _ o o o o o o o # o o # _ _ _
 o # _ _ _ _ o o # # o _ _ o _ _ # _ _ _
 # o # o _ _ # # o _ _ _ _ _ _ _ # _ _ _
 _ o o o _ _ o _ _ _ _ _ _ _ _ _ # o # o
 _ _ _ o # o o _ _ _ _ _ _ _ _ o # o # o
 _ _ _ o # # o _ _ _ _ _ _ _ _ o o _ o o
 _ _ _ _ _ _ _ _ _ _ x x _ _ o o o o # o
 _ _ _ _ x _ _ _ _ _ _ _ _ _ # o # # o o
 _ _ _ _ _ x _ _ _ _ _ _ _ _ o _ o o _ _
 _ _ _ _ _ _ _ _ _ _ _ _ _ _ o o _ _ _ _
 _ _ _ x x x _ _ _ _ _ _ _ _ _ o _ _ _ _
 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ o _ _ _ _
 _ _ x _ _ _ _ o # _ _ _ _ _ _ o # _ _ _
 _ _ _ _ _ _ o o o # o o o o o o o _ _ _



In [17]:
# Define the parameters of the optimization process
params = {"stop_cond": "neval", "neval": 50000, "verbose": True, "v_timer": 2}
pop_size = 100

# Define a population initialization module
pop_init = UniformVectorInitializer(objfunc.vecsize, objfunc.low_lim, objfunc.up_lim, pop_size=pop_size, dtype=int)

# Define the operators to use
mutate_op = OperatorInt("MutSample", {"method": "Uniform", "Low": 0, "Up": 3, "N": 8})
cross_op = OperatorInt("Multipoint")

# Instanciate the algorithm
cro_params = {"rho": 0.8, "Fb": 0.8, "Fd": 0.3, "Pd": 0.2, "attempts": 3}
algorithm = CRO(pop_init, mutate_op, cross_op, cro_params)

# Define a Search method
search = GeneralSearch(objfunc, algorithm, params)

# Optimize the objective function
best_solution, best_fitness = search.optimize()

Initializing optimization of Tresure Hunt using CRO
---------------------------------------------------

Optimizing Tresure Hunt using CRO:
	Real time Spent: 0.03 s
	CPU time Spent:  0.03 s
	Generation: 0
	Best fitness: 10
	Evaluations of fitness: 80

Optimizing Tresure Hunt using CRO:
	Real time Spent: 2.01 s
	CPU time Spent:  2.01 s
	Generation: 51
	Best fitness: 20
	Evaluations of fitness: 4824

Optimizing Tresure Hunt using CRO:
	Real time Spent: 4.03 s
	CPU time Spent:  4.03 s
	Generation: 100
	Best fitness: 24
	Evaluations of fitness: 9412

Optimizing Tresure Hunt using CRO:
	Real time Spent: 6.06 s
	CPU time Spent:  6.06 s
	Generation: 151
	Best fitness: 26
	Evaluations of fitness: 14170

Optimizing Tresure Hunt using CRO:
	Real time Spent: 8.08 s
	CPU time Spent:  8.09 s
	Generation: 201
	Best fitness: 28
	Evaluations of fitness: 18866

Optimizing Tresure Hunt using CRO:
	Real time Spent: 10.11 s
	CPU time Spent:  10.11 s
	Generation: 250
	Best fitness: 30
	Evaluations of fitne

In [18]:
print(f"Number of treasures obtained: {best_fitness}")
objfunc.display_solution(best_solution)

Number of treasures obtained: 36
 o _ _ _ _ _ _ _ _ _ _ o # o o o # o o o
 o _ _ _ _ _ _ _ _ _ _ _ _ _ o _ _ _ o o
 o _ _ _ _ _ _ x _ _ _ _ o o o # _ _ # o
 o _ _ _ _ _ _ _ _ o o o o _ _ _ _ _ o o
 o o _ _ _ _ _ _ _ o o _ _ _ _ _ _ o o _
 _ o _ _ x _ _ _ _ _ o _ _ _ _ _ _ o # o
 _ o _ _ _ _ _ _ o o o _ _ x _ _ # o o _
 o # o _ _ _ _ _ # # _ _ _ _ _ _ # o o _
 # o # _ _ _ # # o o _ _ _ _ _ _ # o o o
 _ o o _ _ o o _ _ o _ _ _ _ _ _ # o # o
 _ _ o o # o _ _ _ _ _ _ _ _ _ _ # o # o
 _ _ _ _ # # o _ _ _ _ _ _ _ _ _ _ _ o o
 _ _ _ _ _ _ o _ o o # # o o _ _ _ _ # _
 _ _ _ _ # o o _ o _ _ _ _ o # o # # o _
 _ _ o o o # o _ o _ _ _ _ _ _ o o _ _ _
 _ _ o o o _ _ o o _ _ _ _ _ _ _ _ _ _ _
 _ _ o # # # o o o _ _ _ _ _ _ _ _ _ _ _
 _ o o o _ _ o o o _ _ _ _ _ _ _ _ _ _ _
 _ _ # o o _ _ o # o _ _ _ _ _ _ x _ _ _
 _ _ _ _ o o o o o # _ _ _ _ _ _ _ _ _ _

